In [89]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses



In [72]:
train_url = "https://github.com/PhilChodrow/PIC16b/blob/master/datasets/fake_news_train.csv?raw=true"

In [73]:
trainData = pd.read_csv(train_url)

In [74]:
trainData

,Unnamed: 0,title,text,fake
0,17366,Merkel: Strong result for Austria's FPO 'big c...,German Chancellor Angela Merkel said on Monday...,0
1,5634,Trump says Pence will lead voter fraud panel,"WEST PALM BEACH, Fla.President Donald Trump sa...",0
2,17487,JUST IN: SUSPECTED LEAKER and “Close Confidant...,"On December 5, 2017, Circa s Sara Carter warne...",1
3,12217,Thyssenkrupp has offered help to Argentina ove...,"Germany s Thyssenkrupp, has offered assistance...",0
4,5535,Trump say appeals court decision on travel ban...,President Donald Trump on Thursday called the ...,0
...,...,...,...,...
22444,10709,ALARMING: NSA Refuses to Release Clinton-Lynch...,If Clinton and Lynch just talked about grandki...,1
22445,8731,Can Pence's vow not to sling mud survive a Tru...,"() - In 1990, during a close and bitter congre...",0
22446,4733,Watch Trump Campaign Try To Spin Their Way Ou...,A new ad by the Hillary Clinton SuperPac Prior...,1
22447,3993,"Trump celebrates first 100 days as president, ...","HARRISBURG, Pa.U.S. President Donald Trump hit...",0


In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cake\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [75]:
def make_dataset(df):
    #
    df["title"] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #
    df["text"] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    
    data = tf.data.Dataset.from_tensor_slices(( 
    #input dictionary
    { "title": df[["title"]],
     "text": df["text"]},

    #output dictionary
    { "fake": df[["fake"]]}
    ))
    return data.batch(100)

In [76]:
dataset = make_dataset(trainData)

In [77]:
dataset_size = len(dataset)
print(dataset_size)

225


# Split Dataset
20% of the dataset is reserved for validation dataset

In [78]:
import random

dataset = dataset.shuffle(buffer_size = dataset_size)

val_size  = int(dataset_size * 0.2)

val = dataset.take(val_size)
train = dataset.skip(val_size).take(dataset_size - val_size)

#check size of training and validation sets
print(len(train), len(val))

180 45


In [82]:

labels_iterator= train.unbatch().map(lambda input, output: output).as_numpy_iterator()
fake = 0
real = 0
for i in labels_iterator:
    if i['fake'] == 1:
        fake +=1
    else:
        real +=1
print(f"{fake} fake and {real} real")
print(f"total sample is {real + fake}")
print(f"Our base rate of fake articles is {fake*100/(fake+real)}%")

9389 fake and 8560 real
total sample is 17949
Our base rate of fake articles is 52.30932085352944%


In [90]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

import re
import string



In [86]:
#preparing a text vectorization layer for tf model
size_vocabulary = 2000

def standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    no_punctuation = tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation),'')
    return no_punctuation

title_vectorize_layer = TextVectorization(
    standardize=standardization,
    max_tokens=size_vocabulary, # only consider this many words
    output_mode='int',
    output_sequence_length=500) 

title_vectorize_layer.adapt(train.map(lambda x, y: x["title"]))


text_vectorize_layer = TextVectorization(
    standardize=standardization,
    max_tokens=size_vocabulary, # only consider this many words
    output_mode='int',
    output_sequence_length=500) 

text_vectorize_layer.adapt(train.map(lambda x, y: x["title"]))

In [91]:
# inputs

title_input = keras.Input(
    shape = (len(scalars),),
    name = "title",
    dtype = "string"
)

NameError: name 'keras' is not defined

In [ ]:
title_features = title_vectorize_layer(title_input)
title_features = layers.Embedding(size_vocabulary, 3, name = "embedding")(lyrics_features)
title_features = layers.Dropout(0.2)(lyrics_features)
title_features = layers.GlobalAveragePooling1D()(lyrics_features)
title_features = layers.Dropout(0.2)(lyrics_features)
title_features = layers.Dense(32, activation='relu')(title_features)

In [ ]:
output = layers.Dense(2, name = "fake")(title_features)

In [ ]:
from tensorflow.keras import utils
utils.plot_model(model)

In [ ]:

text_input = keras.Input(
    shape = (1,),
    name = "text",
    dtype = "string"
)



In [ ]:
text_features = layers.Dense(32, activation='relu')(text_input)